In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import *
from pyspark.sql.functions import broadcast
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 3, Finished, Available)

## **Customer**

In [2]:
# new table
customer_table = spark.sql("SELECT * FROM LTT_SilverLakehouse.customer")
old_customer_table = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_customer")
customer_table.show(5)
old_customer_table.show(5)

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 4, Finished, Available)

+-----------+--------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_segment|
+-----------+--------------+--------------+----------------+
|         12|   Christopher|         Smith|       Corporate|
|         13|          Mary|       Baldwin|     Home Office|
|         16|       Tiffany|         Smith|       Corporate|
|         17|          Mary|      Robinson|        Consumer|
|         19|     Stephanie|      Mitchell|     Home Office|
+-----------+--------------+--------------+----------------+
only showing top 5 rows

+------------+-----------+--------------+--------------+----------------+----------+----------+--------+
|customer_key|customer_id|customer_fname|customer_lname|customer_segment|valid_from|  valid_to|is_valid|
+------------+-----------+--------------+--------------+----------------+----------+----------+--------+
|           1|        210|      Jonathan|         Burns|       Corporate|2024-06-04|9999-12-31|    true|
|     

In [3]:
customer_table.createOrReplaceTempView('customer_table')
old_customer_table.createOrReplaceTempView('old_customer_table')

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 5, Finished, Available)

In [4]:
print(customer_table.count())
print(old_customer_table.count())

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 6, Finished, Available)

1668
33


In [5]:
# Test truong hop change data
fake_data = \
spark.sql("""
    SELECT customer_id, customer_fname, customer_lname, customer_segment
    FROM old_customer_table
    LIMIT 5
""")
display(fake_data)

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1f0a4b0c-c6cf-4699-bc25-831e82ad0b13)

In [6]:
fake_data = fake_data.withColumn("customer_fname", 
                                 when(fake_data["customer_id"] == 1104, "Thang")
                                 .otherwise(fake_data["customer_fname"]))

fake_data = fake_data.withColumn("customer_fname", 
                                 when(fake_data["customer_id"] == 1406, "Tien")
                                 .otherwise(fake_data["customer_fname"]))

fake_data = fake_data.withColumn("customer_fname", 
                                 when(fake_data["customer_id"] == 1837, "Tot")
                                 .otherwise(fake_data["customer_fname"]))

fake_data = fake_data.withColumn("customer_fname", 
                                 when(fake_data["customer_id"] == 1845, "Duc Anh")
                                 .otherwise(fake_data["customer_fname"]))

display(fake_data)
# (210, 1104, 1406, 1837, 1845)

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 06d20030-fc1b-4f1c-8073-487ad22cb039)

In [7]:
fake_data.createOrReplaceTempView('fake_data')
customer_table = customer_table.union(fake_data) # union fake data
customer_table.createOrReplaceTempView('customer_table')
customer_table.count()

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 9, Finished, Available)

1673

In [8]:
new_customer = \
spark.sql("""
    select *
    from customer_table 
    where customer_id not in (
        select customer_id
        from old_customer_table
    )
""")

modified_customer = \
spark.sql("""
    select c.customer_id, c.customer_fname, c.customer_lname, c.customer_segment
    from customer_table c
    join old_customer_table oc
    on c.customer_id == oc.customer_id
    where (c.customer_fname != oc.customer_fname
    or c.customer_lname != oc.customer_lname
    or c.customer_segment != oc.customer_segment)
    and oc.is_valid = 'true'    
""")

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 10, Finished, Available)

In [9]:
print(new_customer.count())
print(modified_customer.count())

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 11, Finished, Available)

1635
4


In [10]:
# combine 2 tables above and scd to them
combined_customer = new_customer.union(modified_customer)
combined_customer = combined_customer.withColumn('valid_from', F.current_date())
combined_customer = combined_customer.withColumn('valid_to', to_date(F.lit('9999-12-31')))
combined_customer = combined_customer.withColumn('is_valid', lit(True))
combined_customer = combined_customer.select('customer_id', 'customer_fname', 'customer_lname', 'customer_segment', 'valid_from', 'valid_to', 'is_valid')

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 12, Finished, Available)

In [11]:
display(combined_customer)

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, c4a4466f-2908-4830-8503-e12dc363f0d5)

In [12]:
# select and update modified table
update_modified = \
spark.sql("""
    select oc.customer_key, oc.customer_id, oc.customer_fname, oc.customer_lname, oc.customer_segment, oc.valid_from, valid_to, is_valid
    from customer_table c
    join old_customer_table oc
    on c.customer_id == oc.customer_id
    where (c.customer_fname != oc.customer_fname
    or c.customer_lname != oc.customer_lname
    or c.customer_segment != oc.customer_segment)
    and oc.is_valid = 'true'    
""")
print(old_customer_table.count())

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 14, Finished, Available)

33


In [13]:
display(update_modified)

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4965abc3-d454-47b1-86e1-c17932114b45)

In [14]:
# eject the modified_table and then append the new update_mofied which is already updated other columns
old_customer_table = old_customer_table.exceptAll(update_modified)
print(old_customer_table.count())
# update_modified.show()

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 16, Finished, Available)

29


In [15]:
old_customer_table.createOrReplaceTempView('old_customer_table')
display(
spark.sql("""
    select *
    from old_customer_table
    where customer_id in (210, 1104, 1406, 1837, 1845)
"""))

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, cc31adea-1f41-4f95-a484-e754685beda0)

In [16]:
# update the column is_valid to false
update_modified = update_modified.withColumn('is_valid', lit(False))
update_modified = update_modified.withColumn('valid_to', current_date())
update_modified.show()

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 18, Finished, Available)

+------------+-----------+--------------+--------------+----------------+----------+----------+--------+
|customer_key|customer_id|customer_fname|customer_lname|customer_segment|valid_from|  valid_to|is_valid|
+------------+-----------+--------------+--------------+----------------+----------+----------+--------+
|           2|       1104|         Linda|       Morales|     Home Office|2024-06-04|2024-06-04|   false|
|           3|       1406|           Amy|         Smith|       Corporate|2024-06-04|2024-06-04|   false|
|           4|       1837|          Mary|          Vega|     Home Office|2024-06-04|2024-06-04|   false|
|           5|       1845|          Mary|         Jones|       Corporate|2024-06-04|2024-06-04|   false|
+------------+-----------+--------------+--------------+----------------+----------+----------+--------+



In [17]:
# get max key and update key for new and update customers
max_key = old_customer_table.selectExpr("max(customer_key)").collect()[0][0]
print(max_key)
# update keys for combined customer
window = Window.orderBy('customer_id')
combined_customer = combined_customer.withColumn('customer_key', max_key + row_number().over(window))

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 19, Finished, Available)

33


In [18]:
# update new keys
combined_customer = combined_customer.select('customer_key', 'customer_id', 'customer_fname', 'customer_lname', 'customer_segment', 'valid_from', 'valid_to', 'is_valid')
combined_customer.show(5)
print(combined_customer.count())

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 20, Finished, Available)

+------------+-----------+--------------+--------------+----------------+----------+----------+--------+
|customer_key|customer_id|customer_fname|customer_lname|customer_segment|valid_from|  valid_to|is_valid|
+------------+-----------+--------------+--------------+----------------+----------+----------+--------+
|          34|         12|   Christopher|         Smith|       Corporate|2024-06-04|9999-12-31|    true|
|          35|         13|          Mary|       Baldwin|     Home Office|2024-06-04|9999-12-31|    true|
|          36|         16|       Tiffany|         Smith|       Corporate|2024-06-04|9999-12-31|    true|
|          37|         17|          Mary|      Robinson|        Consumer|2024-06-04|9999-12-31|    true|
|          38|         19|     Stephanie|      Mitchell|     Home Office|2024-06-04|9999-12-31|    true|
+------------+-----------+--------------+--------------+----------------+----------+----------+--------+
only showing top 5 rows

1639


In [19]:
update_modified.createOrReplaceTempView('update_modified')
combined_customer.createOrReplaceTempView('combined_customer')

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 21, Finished, Available)

In [20]:
display(
spark.sql("""
    select *
    from update_modified
    where customer_id in (210, 1104, 1406, 1837, 1845)
"""))

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 22, Finished, Available)

SynapseWidget(Synapse.DataFrame, e9ffd8c3-94c9-4117-b5f3-f546d2198578)

In [21]:
display(
spark.sql("""
    select *
    from combined_customer
    where customer_id in (210, 1104, 1406, 1837, 1845)
"""))

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 23, Finished, Available)

SynapseWidget(Synapse.DataFrame, 16e02887-34e3-46e9-a7ae-76f8a3d5abec)

In [22]:
# Append the modified_table(false valid) and new_table(true valid + new customer)
old_customer_table = old_customer_table.union(update_modified)
old_customer_table = old_customer_table.union(combined_customer)

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 24, Finished, Available)

In [23]:
display(new_customer.head(3))

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 25, Finished, Available)

SynapseWidget(Synapse.DataFrame, ff1b6359-f80c-4fd6-b374-a5fef198d06f)

In [24]:
# Check the resutl
old_customer_table.createOrReplaceTempView('old_customer_table')
display(
spark.sql("""
    select *
    from old_customer_table
    where customer_id in (210, 1104, 1406, 1837, 1845    , 12, 13)
"""))

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 26, Finished, Available)

SynapseWidget(Synapse.DataFrame, 536bd0f2-3ce0-407a-b842-c1998423856c)

In [25]:
old_customer_table.count()

StatementMeta(, ad251e28-3cce-48d2-8192-28fa9060031d, 27, Finished, Available)

1672

## **Product**


In [ ]:
# new table
product_table = spark.sql("SELECT * FROM LTT_SilverLakehouse.product")
old_product_table = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_product")
product_table.show(5)
old_product_table.show(5)
print(product_table.count())
print(old_product_table.count())

StatementMeta(, , , Cancelled, )

In [ ]:
product_table.createOrReplaceTempView('product_table')
old_product_table.createOrReplaceTempView('old_product_table')

new_product = \
spark.sql("""
    select *
    from product_table 
    where product_id not in (
        select product_id
        from old_product_table
    )
""")

modified_product = \
spark.sql("""
    select p.product_id, p.product_category_id, p.product_name, p.product_price, p.product_status
    from product_table p
    join old_product_table op
    on p.product_id == op.product_id
    where (p.product_category_id != op.product_category_id
        or p.product_name != op.product_name
        or p.product_price != op.product_price
        or p.product_status != op.product_status
    )
    and op.is_valid = 'true'
""")
print(new_product.count())
print(modified_product.count())

StatementMeta(, , , Cancelled, )

In [ ]:
# combine 2 tables above and scd to them
combined_product = new_product.union(modified_product)
combined_product = combined_product.withColumn('valid_from', F.current_date())
combined_product = combined_product.withColumn('valid_to', to_date(F.lit('9999-12-31')))
combined_product = combined_product.withColumn('is_valid', lit(True))
combined_product = combined_product.select('product_id', 'product_category_id', 'product_name', 'product_price', 'product_status', 'valid_from', 'valid_to', 'is_valid')

StatementMeta(, , , Cancelled, )

In [ ]:
# select and update modified table
update_modified = \
spark.sql("""
    select op.product_key, op.product_id, op.product_category_id, op.product_name, op.product_price, op.product_status, op.valid_from, op.valid_to, op.is_valid
    from product_table p
    join old_product_table op
    on p.product_id == op.product_id
    where (p.product_category_id != op.product_category_id
    or p.product_name != op.product_name
    or p.product_price != op.product_price
    or p.product_status != op.product_status)
    and op.is_valid = 'true'    
""")
print(old_product_table.count())

StatementMeta(, , , Cancelled, )

In [ ]:
# eject the modified_table and then append the new update_mofied which is already updated other columns
old_product_table = old_product_table.exceptAll(update_modified)
print(old_product_table.count())
update_modified.show()

StatementMeta(, , , Cancelled, )

In [ ]:
# get max key and update key for new and update records
max_key = old_product_table.selectExpr("max(product_key)").collect()[0][0]
print(max_key)
# update keys for combined customer
window = Window.orderBy('product_id')
combined_product = combined_product.withColumn('product_key', max_key + row_number().over(window))

StatementMeta(, , , Cancelled, )

In [ ]:
combined_product = combined_product.select('product_key', 'product_id', 'product_category_id', 'product_name', 'product_price', 'product_status', 'valid_from', 'valid_to', 'is_valid')
combined_customer.show(5)

StatementMeta(, , , Cancelled, )

In [ ]:
# Append the modified_table(false valid) and new_table(true valid + new customer)
old_product_table = old_product_table.union(update_modified)
old_product_table = old_product_table.union(combined_product)

StatementMeta(, , , Cancelled, )

In [ ]:
print(old_product_table.count())
old_product_table.filter(col('is_valid') == False).show(5) # no changed product

StatementMeta(, , , Cancelled, )

## Product Category Table

In [ ]:
# new table
category = spark.sql("SELECT * FROM LTT_SilverLakehouse.category")
old_category = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_category")
category.show(5)
old_category.show(5)
print(category.count())
print(old_category.count())

StatementMeta(, , , Cancelled, )

In [ ]:
category.createOrReplaceTempView('category')
old_category.createOrReplaceTempView('old_category')

new_category = \
spark.sql("""
    select *
    from category
    where product_category_id not in (
        select product_category_id
        from old_category
    )
""")


modified_category = \
spark.sql("""
    select c.product_category_id, c.category_name, c.department_id
    from category c
    join old_category oc
    on c.product_category_id == oc.product_category_id
    where (c.category_name != oc.category_name 
        or c.department_id != oc.department_id
    )
    and oc.is_valid = 'true'
""")
print(new_category.count())
print(modified_category.count())

StatementMeta(, , , Cancelled, )

In [ ]:
# combine 2 tables above and scd to them
combined_category = new_category.union(modified_category)
combined_category = combined_category.withColumn('valid_from', F.current_date())
combined_category = combined_category.withColumn('valid_to', to_date(F.lit('9999-12-31')))
combined_category = combined_category.withColumn('is_valid', lit(True))
combined_category = combined_category.select('product_category_id', 'category_name', 'department_id', 'valid_from', 'valid_to', 'is_valid')

StatementMeta(, , , Cancelled, )

In [ ]:
# select and update modified table
update_modified = \
spark.sql("""
    select oc.category_key, oc.product_category_id, oc.category_name, oc.department_id, oc.valid_from, oc.valid_to, oc.is_valid
    from category c
    join old_category oc
    on c.product_category_id == oc.product_category_id
    where (c.category_name != oc.category_name 
        or c.department_id != oc.department_id
    )
    and oc.is_valid = 'true'  
""")
print(old_category.count())

StatementMeta(, , , Cancelled, )

In [ ]:
# eject the modified_table and then append the new update_mofied which is already updated other columns
old_category = old_category.exceptAll(update_modified)
print(old_category.count())
update_modified.show()

StatementMeta(, , , Cancelled, )

In [ ]:
# get max key and update key for new and update records
max_key = old_category.selectExpr("max(category_key)").collect()[0][0]
print(max_key)
# update keys for combined customer
window = Window.orderBy('product_category_id')
combined_category = combined_category.withColumn('category_key', max_key + row_number().over(window))

StatementMeta(, , , Cancelled, )

In [ ]:
combined_category = combined_category.select('category_key', 'product_category_id', 'category_name', 'department_id', 'valid_from', 'valid_to', 'is_valid')
combined_category.count()

StatementMeta(, , , Cancelled, )

In [ ]:
# Append the modified_table(false valid) and new_table(true valid + new customer)
old_category = old_category.union(update_modified)
old_category = old_category.union(combined_category)
print(old_category.count())
old_category.filter(col('is_valid') == False).show(5) # no changed product

StatementMeta(, , , Cancelled, )

## Department

In [ ]:
# new table
department = spark.sql("SELECT * FROM LTT_SilverLakehouse.department")
old_department = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_department")
department.show(5)
old_department.show(5)

StatementMeta(, , , Cancelled, )

In [ ]:
print(department.count())
print(old_department.count())

StatementMeta(, , , Cancelled, )

In [ ]:
department.createOrReplaceTempView('department')
old_department.createOrReplaceTempView('old_department')

new_department = \
spark.sql("""
    select *
    from department
    where department_id not in (
        select department_id
        from old_department
    )
""")



modified_department = \
spark.sql("""
    select d.department_id, d.department_name
    from department d
    join old_department od
    on d.department_id = od.department_id
    where (d.department_name != od.department_name)
    and od.is_valid = 'true'
""")
print(new_department.count())
print(modified_department.count())

StatementMeta(, , , Cancelled, )

In [ ]:
# combine 2 tables above and scd to them
combined_department = new_department.union(modified_department)
combined_department = combined_department.withColumn('valid_from', F.current_date())
combined_department = combined_department.withColumn('valid_to', to_date(F.lit('9999-12-31')))
combined_department = combined_department.withColumn('is_valid', lit(True))
combined_department = combined_department.select('department_id', 'department_name', 'valid_from', 'valid_to', 'is_valid')

StatementMeta(, , , Cancelled, )

In [ ]:
# select and update modified table
update_modified = \
spark.sql("""
    select od.department_key, od.department_id, od.department_name, od.valid_from, od.valid_to, od.is_valid
    from department d
    join old_department od
    on d.department_id == od.department_id
    where (d.department_name != od.department_name
    )
    and od.is_valid = 'true'  
""")
print(old_department.count())

StatementMeta(, , , Cancelled, )

In [ ]:
# eject the modified_table and then append the new update_mofied which is already updated other columns
old_department = old_department.exceptAll(update_modified)
print(old_department.count())
update_modified.show()

StatementMeta(, , , Cancelled, )

In [ ]:
# get max key and update key for new and update records
max_key = old_department.selectExpr("max(department_key)").collect()[0][0]
print(max_key)
# update keys for combined customer
window = Window.orderBy('department_id')
combined_department = combined_department.withColumn('department_key', max_key + row_number().over(window))

StatementMeta(, , , Cancelled, )

In [ ]:
combined_department = combined_department.select('department_key', 'department_id', 'department_name', 'valid_from', 'valid_to', 'is_valid')
combined_department.count()

StatementMeta(, , , Cancelled, )

In [ ]:
combined_department.show()

StatementMeta(, , , Cancelled, )

In [ ]:
# Append the modified_table(false valid) and new_table(true valid + new customer)
old_department = old_department.union(update_modified)
old_department = old_department.union(combined_department)


StatementMeta(, , , Cancelled, )

In [ ]:
print(old_department.count())
old_department.filter(col('is_valid') == False).show(5) # no changed product

StatementMeta(, , , Cancelled, )

## Load to Table

In [ ]:
print(old_customer_table.count())
print(old_product_table.count())
print(old_category.count())
print(old_department.count())


StatementMeta(, , , Cancelled, )

In [ ]:
old_customer_table = old_customer_table.orderBy('customer_key')
old_product_table = old_product_table.orderBy('product_key')
old_category = old_category.orderBy('category_key')
old_department = old_department.orderBy('department_key')

StatementMeta(, , , Cancelled, )

In [ ]:
old_customer_table.write.format('delta').mode('overwrite').saveAsTable('LTT_GoldLakehouse.dim_customer')
old_product_table.write.format('delta').mode('overwrite').saveAsTable('LTT_GoldLakehouse.dim_product')
old_category.write.format('delta').mode('overwrite').saveAsTable('LTT_GoldLakehouse.dim_category')
old_department.write.format('delta').mode('overwrite').saveAsTable('LTT_GoldLakehouse.dim_department')

StatementMeta(, , , Cancelled, )